# Coursera IBM Data Science Professional Qualification - Week 2 (Part 2)

``` "Use the Geocoder package or the csv file to create the following dataframe" ```

In [1]:
import pandas as pd
import numpy as np
import pyarrow as pa
import pyarrow.parquet as pq
from io import BytesIO
import zipfile, requests

## Load the dataframe produced in the previous part of this assignment:

In [2]:
filepath = 'week_2_toronto_neighbourhoods.parquet'
df = pq.read_table(filepath).to_pandas()
df.head()

C:\Users\rosst\Anaconda3\lib\site-packages\pyarrow\pandas_compat.py:708: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels = getattr(columns, 'labels', None) or [
C:\Users\rosst\Anaconda3\lib\site-packages\pyarrow\pandas_compat.py:735: FutureWarning: the 'labels' keyword is deprecated, use 'codes' instead
  return pd.MultiIndex(levels=new_levels, labels=labels, names=columns.names)
C:\Users\rosst\Anaconda3\lib\site-packages\pyarrow\pandas_compat.py:752: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels, = index.labels


,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


### As Geocode and URL from assignment do not work, use alternative source:
See: https://www.geonames.org

Request relevant data, unzip it and put it into a pandas dataframe

In [3]:
zipped_file_url = 'https://download.geonames.org/export/zip/CA.zip'
filename = 'CA.txt'

#Request data
r = requests.get(zipped_file_url).content
f = zipfile.ZipFile(BytesIO(r))
data =  BytesIO(f.read('CA.txt'))

#Put data into pandas dataframe
column_names = ['country_code','postal_code','place_name','admin_name1','admin_code1','admin_name2','admin_code2','admin_name3','admin_code3','latitude','longitude','accuracy']
postcode_df = pd.read_csv(data,sep='\t',names=column_names)
postcode_df.head()

,country_code,postal_code,place_name,admin_name1,admin_code1,admin_name2,admin_code2,admin_name3,admin_code3,latitude,longitude,accuracy
0,CA,T0A,Eastern Alberta (St. Paul),Alberta,AB,NaN,NaN,NaN,NaN,54.7660,-111.7174,6.0
1,CA,T0B,Wainwright Region (Tofield),Alberta,AB,NaN,NaN,NaN,NaN,53.0727,-111.5816,6.0
2,CA,T0C,Central Alberta (Stettler),Alberta,AB,NaN,NaN,NaN,NaN,52.1431,-111.6941,5.0
3,CA,T0E,Western Alberta (Jasper),Alberta,AB,NaN,NaN,NaN,NaN,53.6758,-115.0948,5.0
4,CA,T0G,North Central Alberta (Slave Lake),Alberta,AB,NaN,NaN,NaN,NaN,55.6993,-114.4529,6.0


Then remove columns we aren't interested in:

In [4]:
postcode_df.drop(columns=['country_code','admin_name1','admin_code1','admin_name2','admin_code2','admin_name3','admin_code3'],inplace=True)
postcode_df.head()

,postal_code,place_name,latitude,longitude,accuracy
0,T0A,Eastern Alberta (St. Paul),54.7660,-111.7174,6.0
1,T0B,Wainwright Region (Tofield),53.0727,-111.5816,6.0
2,T0C,Central Alberta (Stettler),52.1431,-111.6941,5.0
3,T0E,Western Alberta (Jasper),53.6758,-115.0948,5.0
4,T0G,North Central Alberta (Slave Lake),55.6993,-114.4529,6.0


### Join with existing Neighbourhood data

In [5]:
df = df.merge(postcode_df,left_on='Postal Code',right_on='postal_code',how='left')
df.drop(columns=['postal_code'])
#Check that this has worked:
df.head()

,Postal Code,Borough,Neighbourhood,postal_code,place_name,latitude,longitude,accuracy
0,M3A,North York,Parkwoods,M3A,North York (York Heights / Victoria Village / ...,43.7545,-79.3300,1.0
1,M4A,North York,Victoria Village,M4A,North York (Sweeney Park / Wigmore Park),43.7276,-79.3148,6.0
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",M5A,Downtown Toronto (Regent Park / Port of Toronto),43.6555,-79.3626,6.0
3,M6A,North York,"Lawrence Manor, Lawrence Heights",M6A,North York (Lawrence Manor / Lawrence Heights),43.7223,-79.4504,6.0
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",M7A,Queen's Park Ontario Provincial Government,43.6641,-79.3889,NaN


### Output this dataframe as a Parquet file for use in part 3 of the assignment

In [6]:
table = pa.Table.from_pandas(df)
pq.write_table(table,'week_2_part2_data.parquet')

print('Saved dataframe in parquet format for ease of use in next assignment')
print('For more information, see: https://arrow.apache.org/docs/python/parquet.html')

Saved dataframe in parquet format for ease of use in next assignment
For more information, see: https://arrow.apache.org/docs/python/parquet.html


C:\Users\rosst\Anaconda3\lib\site-packages\pyarrow\pandas_compat.py:114: FutureWarning: A future version of pandas will default to `skipna=True`. To silence this warning, pass `skipna=True|False` explicitly.
  result = infer_dtype(pandas_collection)


## Compare the results against the lines in the example on Coursera

In [7]:
list_on_coursera = ['M5G','M2H','M4B','M1J','M4G','M4M','M1R','M9V','M9L','M5V','M1B','M5A']
check_df = df[df['Postal Code'].isin(list_on_coursera)].set_index('Postal Code')
check_df.loc[list_on_coursera,['Borough','Neighbourhood','latitude','longitude']]

,Borough,Neighbourhood,latitude,longitude
Postal Code,,,,
M5G,Downtown Toronto,Central Bay Street,43.6564,-79.3860
M2H,North York,Hillcrest Village,43.8015,-79.3577
M4B,East York,"Parkview Hill, Woodbine Gardens",43.7063,-79.3094
M1J,Scarborough,Scarborough Village,43.7464,-79.2323
M4G,East York,Leaside,43.7124,-79.3644
M4M,East Toronto,Studio District,43.6561,-79.3406
M1R,Scarborough,"Wexford, Maryvale",43.7507,-79.3003
M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.7432,-79.5876
M9L,North York,Humber Summit,43.7598,-79.5565
